In [ ]:
%matplotlib inline
import flopy
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pathlib as pl

## Load the existing model

In [ ]:
# load the simple model setup

name = "sv"
existing_ws = pl.Path("../models/synthetic-valley-vg")
ws = existing_ws.parent / "synthetic-valley-vg-gwt"

In [ ]:
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=existing_ws, write_headers=False
)

In [ ]:
gwf = sim.get_model(name)
nlay, ncpl = gwf.disv.nlay.array, gwf.disv.ncpl.array
cell2d = gwf.disv.cell2d.array
nvert = gwf.disv.nvert.array
vertices = gwf.disv.vertices.array
shape3d = (nlay, ncpl)

In [ ]:
nper = sim.tdis.nper.array
perlen = sim.tdis.perioddata.array["perlen"]

Lets look at the base model using FloPy

### Change the simulation workspace

In [ ]:
sim.set_sim_path(ws)

### Add the groundwater transport model

In [ ]:
gwt_name = f"{name}_gwt"

In [ ]:
gwt = flopy.mf6.ModflowGwt(
    sim,
    modelname=gwt_name,
)


Create the gwt disv package fron gwf disv data

Create gwt ic file with initial concentration of 0.0

Create the advection file. Use the TVD or UTVD scheme

Add the dispersion package (DSP) with alphah = 75, ath1=7.5, and diffc=0

Add the mst package with porosity equal to the gwf sy, zero_order_decay=True, and decay=-1/year

Add the ssm package with no variables except gwt

In [ ]:
oc = flopy.mf6.ModflowGwtoc(
    gwt,
    concentration_filerecord=f"{name}.ucn",
    saverecord={
        1: [
            ("CONCENTRATION", "LAST"),
        ]
    },
    printrecord=[("BUDGET", "ALL")],
)




Add the mover package (mvt) since movers are turned on in the base model

Add the IMS package

In [ ]:
# add ims for
ims = flopy.mf6.ModflowIms(
    sim,
    filename=f"{gwt.name}.ims",
    print_option="all",
    complexity="simple",
    outer_maximum=200,
    inner_maximum=100,
    linear_acceleration="bicgstab",
)
sim.register_ims_package(ims, model_list=[f"{gwt.name}"])




Add the GWF-GWT exchange

In [ ]:
# add gwf-gwt exchange
exchange = flopy.mf6.ModflowGwfgwt(
    sim,
    exgtype="GWF6-GWT6",
    exgmnamea=gwf.name,
    exgmnameb=gwt.name,
)

### Write and run the simulation

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

## Plot the results

In [ ]:
times = gwt.output.concentration().get_times()
conc = gwt.output.concentration().get_data(totim=times[-1])
conc[conc < 1e30].max()

In [ ]:
six_panel_figsize = (17.15 / 2.541, 1.4 * 0.8333 * 17.15 / 2.541)
mosaic = [[0, 1, 2], [3, 4, "."]]

In [ ]:
contour_color = "white"
contour_style = "--"

sv_gwt_contour_dict = {
    "linewidths": 0.75,
    "colors": contour_color,
    "linestyles": contour_style,
}

In [ ]:
with flopy.plot.styles.USGSMap():
    fix, axd = plt.subplot_mosaic(
        mosaic,
        figsize=six_panel_figsize,
        layout="constrained",
    )

    times = gwt.output.concentration().get_times()
    conc = gwt.output.concentration().get_data(totim=times[-1])

    for k in range(nlay):
        ax = axd[k]
        ax.set_xlim(gwf.modelgrid.extent[:2])
        ax.set_ylim(gwf.modelgrid.extent[2:])
        ax.set_title(f"Layer {k + 1}")
        mm = flopy.plot.PlotMapView(
            model=gwt,
            ax=ax,
            extent=gwt.modelgrid.extent,
            layer=k,
        )
        norm = mpl.colors.LogNorm(vmin=1, vmax=30)
        mm.plot_array(conc, vmin=5, vmax=25, edgecolor="black", lw=0.25)

        # mm.plot_grid(lw=0.5, color="0.75")
        mm.plot_bc(package=gwf.sfr, color="cyan")
        mm.plot_bc(package=gwf.lak)
        mm.plot_bc(package=gwf.wel, color="red")

        cs = mm.contour_array(
            conc,
            masked_values=[0],
            levels=np.arange(10, 30, 2.5),
            **sv_gwt_contour_dict,
        )

        ax.clabel(
            cs,
            inline=True,
            fmt="%2.1f",
            fontsize=8,
            inline_spacing=0.5,
            colors="white",
        )